# Accurate $p$ Estimation — 10k Measurements in 5 Seconds

**Key features**:
- Robustness:
    - Works even if numerical integration fails or the system is chaotic;
    - Handles measurements with unequal time steps;
    - Allow less shooting nodes than the measurements;
- Effectiveness: 
    - Shooting in parallel;
    - Solve sparse QP with Schur complement;
    - `ipopt` with precomputed $H=J^\top J$ and `JIT`.

## Plan vs Actual

**Plan**: One week per Milestone.

**Actual**:
- Week 1: 
    - Environment (all)
    - Papers (all)
    - Code framework (Jiaze Li)
    - Start `_build_cnlls` (Heyuan Chi) and `_solve_ipopt` (Jiaze Li)
- Week 2:
    - Start `examples.py` (Jiaze Li)
    - Start `_solve_gn` (Heyuan Chi)
- Week 3: 
    - Gauss-Newton sometimes does not converge, add line search (Heyuan Chi)
    - Start `demo.ipynb` (Jiaze Li)
- Week 4: Realized that shooting can be **parallel** and $H$ is **sparse**
    - Refactored `_build_cnlls` (Jiaze Li)
    - Add Schur complement in `_solve_gn` (Jiaze Li)

*In \(\*\) are the people who mainly wrote the code, but the math are discussed by all of us*

## Some demo

In [ ]:
import numpy as np
import dms_estimator as dmse
from matplotlib import pyplot as plt

%matplotlib inline
plt.rc('figure', dpi=100)
np.set_printoptions(linewidth=100, suppress=True, precision=2, floatmode='fixed')

###  Effectiveness of multiple shooting

Notorious test in §6 of [[Bock07](https://onlinelibrary.wiley.com/doi/pdf/10.1002/gamm.200790024?casa_token=1vkAqyG6JuwAAAAA%3AGwUN3aX6DOsp_FOatMYMQSZCDm27CWARJg90nhxYuZE_UCYlOsUxKim3UcWdrZjMtffexjzbiOGRJA)]

ODE:
$$
\dot{x}_1=x_2 \quad \dot{x}_2=\mu^2 x_1-\left(\mu^2+p^2\right) \sin p t \quad t \in[0,1]
$$

Initial values:
$$
x_1(0)=0 \quad x_2(0)=\pi
$$

The solution for the true parameter value $p=\pi$

$$
x_1(t)=\sin \pi t, \quad x_2(t)=\pi \cos \pi t
$$

In [ ]:
# Setup notorious test
cfg = dmse.ModelConfig(
        name="Notorious",
        build_integrator=dmse.notorious_problem,
        true_p=[np.pi],
        x0=np.array([0.0, np.pi, 0.0]),
        p_init=[1.0],
        state_labels=["x1", "x2", "t"],
)

Let's see what happens when we do numerical integration from $0$

In [ ]:
# Comparing numerical and analytical solutions
dt = 0.01
t_grid = np.arange(0.0, 1.0, dt)

integrator, ode, states, params = cfg.build_integrator(dt)
X_true, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=0.0)
plt.plot(t_grid, X_true[:, 0], label='cvodes solution')

dt = 0.01
t_grid = np.arange(0.0, 1.0, dt)

X_true = np.array([
    np.sin(np.pi * t_grid), 
    np.pi * np.cos(np.pi * t_grid), 
    t_grid]
    ).T
rng = np.random.default_rng(42)
plt.plot(t_grid, X_true[:, 0], label='analytic solution')

plt.ylabel("$x_1$")
plt.ylim(0, 2)
plt.legend()
plt.show()

In [ ]:
# Generate data with analytic solution
dt = 0.1
t_grid = np.arange(0.0, 1.0, dt)

X_true = np.array([
    np.sin(np.pi * t_grid), 
    np.pi * np.cos(np.pi * t_grid), 
    t_grid]
    ).T
rng = np.random.default_rng(42)
X_meas = X_true + 0.01 * rng.standard_normal(X_true.shape)

In [ ]:
print("[Single Shooting]")
p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_true,
    cfg.p_init,
    num_shooting=1,
    strategy="gn_fast",
)
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

Single shooting failed, try multiple shooting

In [ ]:
print("[Multiple Shooting]")
p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_meas,
    cfg.p_init,
    num_shooting=len(t_grid),
    strategy="gn",
)
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

Even for [**chaotic**](https://en.wikipedia.org/wiki/Lorenz_system#) dynamic system!

$$
\begin{aligned}
& \dot{x}=\sigma(y-x), \\
& \dot{y}=x(\rho-z)-y, \\
& \dot{z}=x y-\beta z .
\end{aligned}
$$

In [ ]:
# Try Lorenz system
cfg = dmse.ModelConfig(
        name="Lorenz",
        build_integrator=dmse.lorenz_problem,
        true_p=[10.0, 28.0, 8.0/3.0],
        x0=np.array([3.0, 1.5, 15.0]),
        p_init=[10.0, 30.0, 3.0],
        state_labels=["x", "y", "z"],
)

dt = 0.02
t_grid = np.arange(0.0, 10.0, dt)

integrator, ode, states, params = cfg.build_integrator(dt)
X_true, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=0.0)

p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_meas,
    cfg.p_init,
    num_shooting=500,
    strategy="gn_fast",
)
dmse.plot(t_grid, X_meas[:, :2], s_hat[:, :2], cfg.state_labels[:2], true=X_true[:, :2])
plt.show()
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

### Solve the pyridine problem

In [ ]:
# Pyridine
from ipywidgets import interact, IntSlider, Dropdown

cfg = dmse.ModelConfig(
        name="Pyridine",
        build_integrator=dmse.pyridine_problem,
        true_p=[1.81, 0.894, 29.4, 9.21, 0.058, 2.43, 0.0644, 5.55, 0.0201, 0.577, 2.15],
        x0=np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        p_init=[1.0] * 11,
        state_labels=list("ABCDEFG"),
)

T = 5.0
dt = T / 500
t_grid = np.arange(0.0, T, dt)
integrator, ode, states, params = cfg.build_integrator(dt)
_, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=1e-2)
# Perform estimation
p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_meas,
    cfg.p_init,
    num_shooting=500,
    strategy="gn_fast",
)
# Plot results
dmse.plot(t_grid, X_meas, s_hat, cfg.state_labels, show_every=5)
plt.show()
print(p_hat)
print("True parameters:")
print(np.array(cfg.true_p))

$p_4$ is the coefficient of $C^2$ and $p_8, p_{11}$ if the coefficient of E

$C$ and $E$ are almost $0$, so larger error for these three.

## Code Structure

```
dms_estimator/
├── __init__.py
├── utils.py
├── parameter_estimator.py
└── interface.py
```

Core: `parameter_estimator.py`

| Method                 | What it does                                                 |
| ---------------------- | ------------------------------------------------------------ |
| **Static helpers**     | *Pure utility*, not tied to an instance. <br/>`_select_jit()` → decide whether CasADi’s JIT compiler is available (clang / shell). <br/>`_has_hsl()` → check for sparse HSL linear solvers (MA27, etc.). |
| **`__init__()`**       | Stores user data (ODE, symbols, measurements, options). <br/>Determines the number of shooting nodes and issues warnings if it differs from the number of measurements. <br/>Calls `_build_cnlls()` to create the constrained nonlinear least-squares (CNLLS) problem. |
| **`_build_cnlls()`**   | Heart of the multiple-shooting setup.<br/>1. Computes measurement time steps `Δt`.<br/>2. Constructs a *single-step* RK4 integrator function `one_step_dt`.<br/>3. Creates symbolic shooting-node state matrix `Xv` and maps the integrator across all intervals in parallel (`map("thread")`).<br/>4. Handles two cases: one node = one measurement (simpler) vs. fewer nodes than measurements .<br/>5. Produces:<br/>  • Decision vector `self.variables = [p, Xv]`<br/>  • Residual vector between predicted & measured states (`self.errors`)<br/>  • Continuity constraints `self.constrain` (gaps ≡ 0)<br/>  • Initial guess `self.x0`. |
| **`solve()` (public)** | Simple dispatcher that calls one of three back-end solvers based on the chosen strategy string. |
| **`_solve_ipopt()`**   | • Builds a *full-space* nonlinear program *f = ½‖e‖²; g = gaps* and calls CasADi’s **IPOPT** backend. <br/>• Uses IPOPT’s own quasi-Newton Hessian (unless the user enables exact). |
| **`_solve_gn_fast()`** | *Gauss-Newton Hessian + IPOPT*<br/>1. Symbolically forms Jacobian `J = ∂e/∂w`.<br/>2. Supplies IPOPT with an exact Gauss-Newton Hessian H = JᵀJ via a lightweight callback (`hess_lag`). Only its upper triangle is stored to exploit symmetry. |
| **`_solve_gn()`**      | *Pure, Gauss-Newton with qpoases*<br/>1. Builds functions for residuals, constraints, their Jacobians, and objective.<br/>2. Outer loop: up to `max_iter`, evaluate residuals, assemble H = JᵀJ and g = Jᵀr.<br/>3. Inner QP (`solve_qp` helper): solves. Implemented with CasADi’s `qpoases` wrapper; uses the HSL Schur option when available.<br/>4. Back-tracking Armijo line-search ensures objective decrease.<br/>5. Terminates on small relative residual improvement or stagnation. |

Check out Christian Kirches' presentation [A sparse variant of qpOASES](https://www.syscop.de/files/2014ss/imtek-tempo/EQP_2014_Kirches.pdf) at EQP2014 to learn how Schur complementary works.

## Benchmark Test

Time(s) consumed for different \#measurements, \#nodes and strategies(from `benchmark.py`)

![table](./assets/table.svg)

\*all converge to acceptable result

where
- **IP** is `ipopt`
- **IG** is `ipopt` with the given $H = J^\top J$ and `JIT`
- **GN** is the Gauß-Newton with Schur complement.

**Findings**:
- **GN**: Gauß-Newton is optimal when the `#node` is small;
- **IPs**: The complexity of the `ipopt` does not increase significantly with the number of decision variables;
- **IG**: Outstanding in large-scale. Possible reasons:
    1. Using $J^\top J$ saves time in computing the Hessian and is good enough in least squares problems;
    2. `JIT` compiles the solver firstly thus saving time for each iteration.

**Problems**:
- I don't think the complexity of `qpoases` here should be more than $\mathcal{O}(n)$, but time-wise it's close to this.